In [ ]:
import matplotlib.pyplot as plt
import polars as pl
import util
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
import math

In [ ]:
df = util.load_data().fetch(1_000_000)
# df = util.load_data().collect()

# Run preprocess.py to obtain the parquet dataset
# df = pl.read_parquet('datasets/train.parquet')
df.head()

In [ ]:
df.shape

## Data distribution

In [ ]:
# NB: This plot takes a lot of time
util.plot_distributions(df)

In [ ]:
plt.hist(df['fare_amount'], bins=60, range=(0, 60))
plt.title('Closeup - fare amount')
plt.show()

In [ ]:
plt.hist(df['pickup_latitude'], bins=60, range=(40.65, 40.85))
plt.title('Closeup - pickup latitude')
plt.show()

In [ ]:
plt.hist(df['pickup_longitude'], bins=60, range=(-74.1, -73.75))
plt.title('Closeup - pickup longitude')
plt.show()

In [ ]:
plt.hist(df['pickup_datetime'], bins=7)
plt.title('Closeup - Timestamp')
plt.show()

The distributions suggest the existence of unrealistic data (noise?) and outliers (hundres of passengers for one run, thosands of dollars for a single run). Min and max values show this very clearly. Before moving on with other statistics, it may be a good idea to clear the data further.

In [ ]:
df.describe()

### Passenger count
According to the [NYC taxi commission](https://www.nyc.gov/site/tlc/passengers/passenger-frequently-asked-questions.page#:~:text=The%20maximum%20amount%20of%20passengers,of%20an%20adult%20passenger%20seated) the maximum number of passengers, for suitable vehicles, is five. An additional sixth person (child) is admitted. Thus, it is possible to consider all samples that exceed the number of six passengers to be noise. In fact, values greater than six are highly underrepresented.

In [ ]:
df.groupby('passenger_count').agg(pl.count()).sort('passenger_count')

In [ ]:
df = df.filter(pl.col('passenger_count') <= 6)

## Analyzing spatial locations

In [ ]:
# train, test = train_test_split(df, test_size=0.2)
# train, valid = train_test_split(train, test_size=0.2)

In [ ]:
def print_point_on_map(ax, x, y, points_area, image, markersize=.5, color='b', title=None):
    left, right, bottom, top = points_area
    ax.imshow(image, extent=(left, right, bottom, top))
    ax.set_ylim(bottom, top)
    ax.set_xlim(left, right)
    ax.scatter(x, y, markersize, color)
    if title is not None:
        ax.title.set_text(str(title))

In [ ]:
x = df['pickup_longitude'].append(df['dropoff_longitude'])
y = df['pickup_latitude'].append(df['dropoff_latitude'])
points_area = x.min(), x.max(), y.min(), y.max()

# Make the area a square
width = util.distance((points_area[0],points_area[2]), (points_area[1],points_area[2]))
height = util.distance((points_area[0],points_area[2]), (points_area[0],points_area[3]))

additional_space = (width - height)/2

new_lat_min, _ = util.find_latitude_correction((points_area[0],points_area[2]), additional_space, b=-1)
new_lat_max, _ = util.find_latitude_correction((points_area[0],points_area[3]), additional_space, b=1)

points_area = points_area[0], points_area[1], new_lat_min, new_lat_max
# print(util.distance((points_area[0],points_area[2]), (points_area[1],points_area[2])))
# print(util.distance((points_area[1],points_area[2]), (points_area[1],points_area[3])))
# print(util.distance((points_area[1],points_area[3]), (points_area[0],points_area[3])))
# print(util.distance((points_area[0],points_area[3]), (points_area[0],points_area[2])))

url = 'https://b.basemaps.cartocdn.com/light_nolabels/{z}/{x}/{y}.png'
image = util.new_york_map(points_area)

plt.imshow(image)
plt.show()
print(points_area)

In [ ]:
# Remove points on ocean, not working at the moment
ocean_pickup = df.select(
    pl.struct(['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'])
    .map(util.polars_point_on_ocean(points_area, pickup=True))
    ).get_columns()[0].alias('ocean_pickup')
print("pickup done")
ocean_dropoff = df.select(
    pl.struct(['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'])
    .map(util.polars_point_on_ocean(points_area, dropoff=True))
    ).get_columns()[0].alias('ocean_dropoff')

print('Pickups in the ocean', ocean_pickup.arg_true().shape[0])
print('Dropoffs in the ocean', ocean_dropoff.arg_true().shape[0])
print('Total ocean outlier samples',
      (ocean_dropoff | ocean_pickup).arg_true().shape[0])

outsiders_pickup = df.filter(ocean_pickup)
outsiders_dropoff = df.filter(ocean_dropoff)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(30, 30))

print_point_on_map(axs[0], outsiders_pickup['pickup_longitude'], outsiders_pickup['pickup_latitude'], points_area, image, color='b', markersize=3)
print_point_on_map(axs[1], outsiders_dropoff['dropoff_longitude'], outsiders_dropoff['dropoff_latitude'], points_area, image, color='r', markersize=3)

In [ ]:
df = df.filter(~ocean_pickup & ~ocean_dropoff)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(50, 50))
print_point_on_map(axs[0], df['pickup_longitude'], df['pickup_latitude'], points_area, image, color='b')
print_point_on_map(axs[1], df['dropoff_longitude'], df['dropoff_latitude'], points_area, image, color='r')

In [ ]:
timezone = -5
lavorative_hours = (8, 18)

day_hours = df.filter((pl.col("pickup_datetime").dt.hour() > lavorative_hours[0]+timezone) & (pl.col("pickup_datetime").dt.hour() < lavorative_hours[1]+timezone))
night_hours = df.filter((pl.col("pickup_datetime").dt.hour() <= lavorative_hours[0]+timezone) | (pl.col("pickup_datetime").dt.hour() >= lavorative_hours[1]+timezone))

print(len(day_hours), len(night_hours))

x_day = day_hours['pickup_longitude'].append(day_hours['dropoff_longitude'])
y_day = day_hours['pickup_latitude'].append(day_hours['dropoff_latitude'])

x_night = night_hours['pickup_longitude'].append(night_hours['dropoff_longitude'])
y_night = night_hours['pickup_latitude'].append(night_hours['dropoff_latitude'])

fig, axs = plt.subplots(1, 2, figsize=(16, 16))
print_point_on_map(axs[0], x_day, y_day, points_area, image, color='b', markersize=0.1)
print_point_on_map(axs[1], x_night, y_night, points_area, image, color='r', markersize=0.1)

In [ ]:
hours = []

for h in range(5,29):
    hour_df = df.filter(pl.col("pickup_datetime").dt.hour() == h % 24)
    hours.append((hour_df['pickup_longitude'].append(hour_df['dropoff_longitude']),
                  hour_df['pickup_latitude'].append(hour_df['dropoff_latitude']),
                  len(hour_df)))
    
fig, axs = plt.subplots(6, 4, figsize=(16, 20))
for h in range(24):
    print_point_on_map(axs[h//4, h % 4], hours[h][0], hours[h][1], points_area, image, color='b',
                       title=f'{(h) % 24}. {hours[h][2]} rides', markersize=0.05)

## Time
Specific features have to be extracted from timestamps, based on periods and trends.

### Monthly-Yearly trend

The first thing to investigate is the presence of trends, as detrending data will eventually be necessary before looking for periods. Public transportation rarely becomes cheaper, (at least, that how it is in Bologna). The NYC taxi data spans for about six years, hence, let us look for a yearly trend

In [ ]:
plt.scatter(
    *df.select(['pickup_datetime', 'fare_amount']).sort('pickup_datetime')
    .groupby_dynamic('pickup_datetime', every='1y')
    .agg(pl.mean('fare_amount'))
    .get_columns())
plt.title('Yearly average fare')
plt.show()

In [ ]:
df.select(['pickup_datetime', 'fare_amount']).sort('pickup_datetime') \
  .groupby_dynamic('pickup_datetime', every='1y') \
  .agg(pl.col('fare_amount').std().alias('std'))

Even though variance is pretty high, there is an unambiguous yearly trend (NYC is not that different from Bologna after all). In fact, it is a well known fact that public transportation fares in NYC have been adjusted over time to deal with inflation [2]. In particular, a notable increase happened between 2012 and 2013.

In [ ]:
plt.scatter(
    *df.select(['pickup_datetime', 'fare_amount']).sort('pickup_datetime')
    .filter(pl.col('pickup_datetime').dt.year() == 2012)
    .groupby_dynamic('pickup_datetime', every='1mo')
    .agg(pl.mean('fare_amount'))
    .get_columns())
plt.title('Per-month average fares (2012)')
plt.show()

Further inspection shows that the gap happened in September 2012. In fact, historical record witnesses this. According to the *Fare and Lease Cap Report* of April 2013 [3], during fall 2012 fares have increased by 17%, apparently in order to handle a change in credit card processing fees.

Approximating the trend is mandatory in order to proceed with the inspection of periods. Given its nature (inflation) it is acceptable to consider it linear. However, the big gap of September 2012 can be considered anomalous and would clearly skew the linear coefficients. For this reason, two linear trends are considered, with a discontinuity exactly in September 2012.

In [ ]:
# Compute the monthly average fares and split them before/after Sep 2012
months_df = (
    df.select(['pickup_datetime', 'fare_amount'])
    .sort('pickup_datetime')
    .groupby_dynamic('pickup_datetime', every='1mo')
    .agg(pl.mean('fare_amount'))
)

months_pre2012_df = (
    months_df.filter(pl.col('pickup_datetime') < pl.datetime(2012, 9, 1))
)

months_post2012_df = (
    months_df.filter(pl.col('pickup_datetime') >= pl.datetime(2012, 9, 1))
)

# Sanity check on the dataframe shapes
split_month_span = len(months_pre2012_df) + len(months_post2012_df)
assert split_month_span == len(months_df)

print('Total months', split_month_span)
print('Months before Sep 2012:', len(months_pre2012_df))
print('Months after Sep 2012:', len(months_post2012_df))

In [ ]:
# Fit two OLS regressors
months_features = np.array(range(len(months_df))).reshape(-1, 1)
months_pre2012_features = months_features[:len(months_pre2012_df)]
months_post2012_features = months_features[len(months_pre2012_df):]

trend_pre2012_regressor = LinearRegression(n_jobs=-1).fit(
    months_pre2012_features, months_pre2012_df['fare_amount'])
trend_post2012_regressor = LinearRegression(n_jobs=-1).fit(
    months_post2012_features, months_post2012_df['fare_amount'])

In [ ]:
trend_pre2012_pred = trend_pre2012_regressor.predict(months_pre2012_features)
trend_post2012_pred = trend_post2012_regressor.predict(months_post2012_features)

plt.figure(figsize=(40, 8))
plt.plot(*months_df.get_columns())
plt.plot(months_df['pickup_datetime'],  np.concatenate((trend_pre2012_pred, trend_post2012_pred)))
plt.legend(('Monthly average fare', 'Linear average fare'), fontsize='xx-large')
plt.show()

The double-linear regression seems to be quite reasonable. This shall result in the extraction of a couple of dedicated features from the timestamps. However, which ones exactly will be discussed after an analysis on periods.

### Periods
A detrended/stationary series can now be computed in order to have a cleaner overview of the periods.

In [ ]:
# TODO: Detrend before plotting

plt.figure(figsize=(40, 8))
plt.plot(
    *df.select(['pickup_datetime', 'fare_amount']).sort('pickup_datetime')
    .groupby_dynamic('pickup_datetime', every='1mo')
    .agg(pl.mean('fare_amount'))
    .get_columns())
plt.show()

In [ ]:
jfk_position = (40.653005, -73.797447)
manhattan_position = (40.722433, -74.000845)
precision = 0.01

df_grt_2013 = df.filter((pl.col('pickup_datetime').is_between(pl.datetime(2013, 1, 1), pl.datetime(2100, 1, 1))))
df_grt_2013 = df_grt_2013.with_column(pl.col("pickup_datetime").dt.hour().alias('pickup_hour'))

df_manhattan = df_grt_2013.filter(((pl.col('pickup_latitude').is_between(manhattan_position[0] - precision, manhattan_position[0] + precision, closed='both')) &
                                 (pl.col('pickup_longitude').is_between(manhattan_position[1] - precision, manhattan_position[1] + precision, closed='both'))) |
                                 ((pl.col('dropoff_latitude').is_between(manhattan_position[0] - precision, manhattan_position[0] + precision, closed='both')) &
                                 (pl.col('dropoff_longitude').is_between(manhattan_position[1] - precision, manhattan_position[1] + precision, closed='both'))))

df_jfk = df_grt_2013.filter(((pl.col('pickup_latitude').is_between(jfk_position[0] - precision, jfk_position[0] + precision, closed='both')) &
                                 (pl.col('pickup_longitude').is_between(jfk_position[1] - precision, jfk_position[1] + precision, closed='both'))) |
                                 ((pl.col('dropoff_latitude').is_between(jfk_position[0] - precision, jfk_position[0] + precision, closed='both')) &
                                 (pl.col('dropoff_longitude').is_between(jfk_position[1] - precision, jfk_position[1] + precision, closed='both'))))


print(f"Number of samples from or to jfk: {len(df_jfk)}")
print(f"Number of samples from or to manhattan: {len(df_manhattan)}")

fg, axs = plt.subplots(2,1, figsize=(40, 8))
axs[0].hist(df_manhattan['pickup_hour'], bins=24)
axs[0].title.set_text('Hour samples distribution from or to manhattan')

axs[1].plot(
    *df_manhattan.select(['pickup_hour', 'fare_amount'])
    .groupby('pickup_hour')
    .agg(pl.mean('fare_amount'))
    .sort('pickup_hour')
    .get_columns())
axs[1].title.set_text("Mean hourly fare from or to manhattan")

plt.show()

fg, axs = plt.subplots(2,1, figsize=(40, 8))

axs[0].hist(df_jfk['pickup_hour'], bins=24)
axs[0].title.set_text('Hour samples distribution from or to jfk')

axs[1].plot(
    *df_jfk.select(['pickup_hour', 'fare_amount'])
    .groupby('pickup_hour')
    .agg(pl.mean('fare_amount'))
    .sort('pickup_hour')
    .get_columns())
axs[1].title.set_text("Mean hourly fare from or to jfk")

plt.show()

In [ ]:
import importlib
importlib.reload(util)

# References
*TODO: properly cite?*

[2]: Why Subway and Bus Fares Are Likely to Rise Next Year, https://www.nytimes.com/2022/12/19/nyregion/why-subway-and-bus-fares-are-likely-to-rise-next-year.html

[3]: Fare and Lease Cap Report: April 2013, https://a860-gpp.nyc.gov/concern/nyc_government_publications/jm214q472?locale=en

[4]: norta - code for New Orleans Regional Transit Authority Data, https://git.bryanbrattlof.com/norta/about/